In [1]:
import pandas as pd
import numpy as np
import sys,os
sys.path.append(os.path.expanduser("~")+"/ivpy/src")
from ivpy import *
from ivpy.extract import norm
import warnings
warnings.filterwarnings('ignore')

In [2]:
sys.path.append(os.path.expanduser("~")+"/collproc")
from coll import *

#### roughness

In [3]:
rf = pd.read_csv("/Users/damoncrockett/lmlproc/proc/texture/genome_roughness_768_250_501_21_divide.csv")

In [4]:
rf['catalog'] = [os.path.basename(item)[:-4].split("_")[0] for item in rf.localpath]

In [5]:
def rng(col, return_bounds=False):
    
    colmin = min(col)
    colmax = max(col)
    
    if return_bounds:
        return colmax-colmin, colmin, colmax
    
    return colmax - colmin

In [6]:
roughness_ranges = rf.groupby('catalog').agg(rng).reset_index().roughness

#### dmin

In [7]:
cf = pd.read_csv("/Users/damoncrockett/lmlproc/proc/color/_OLD/colors.csv")

In [8]:
cf = cf[['cat','mloc','LAB_B','LAB_L']][cf.mloc!='dmax']

In [9]:
cf = cf[cf.LAB_L>=75]

In [10]:
dmin_ranges = cf[['cat','LAB_B']].groupby('cat').agg(rng).reset_index().LAB_B

#### dmax

In [11]:
cf = pd.read_csv("/Users/damoncrockett/lmlproc/proc/color/_OLD/colors.csv")

In [12]:
cf = cf[['cat','mloc','LAB_B','LAB_L']][cf.mloc=='dmax']

In [13]:
cf = cf[cf.LAB_L<=48]

In [14]:
dmax_ranges = cf[['cat','LAB_B']].groupby('cat').agg(rng).reset_index().LAB_B

#### gloss

In [15]:
gf = pd.read_csv("/Users/damoncrockett/lmlproc/proc/gloss/_OLD/gloss_disagg.csv")

In [16]:
gf = gf[gf.g60 < 114]

In [17]:
gloss_ranges = gf[['catalog','g60']].groupby('catalog').agg(rng).reset_index().g60

#### thickness

In [18]:
tf = pd.read_csv("/Users/damoncrockett/lmlproc/proc/thickness/_OLD/thickness_disagg.csv")

In [19]:
from ast import literal_eval

In [20]:
tf.measurements = tf.measurements.apply(literal_eval)

In [21]:
tf.measurements = tf.measurements.apply(lambda x: [abs(item) for item in x if abs(item) < 1])

In [22]:
tf['tlen'] = tf.measurements.apply(len)

In [23]:
tf = tf[tf.tlen > 0]

In [24]:
tf['mrange'] = tf.measurements.apply(rng)

In [25]:
thickness_ranges = tf.mrange

# Range dict

In [26]:
rd = {}

In [27]:
rd['thickness'] = [round(item,4) for item in list(thickness_ranges)]
rd['gloss'] = [round(item,4) for item in list(gloss_ranges)]
rd['roughness'] = [round(item,4) for item in list(roughness_ranges)]
rd['bstar_base'] = [round(item,4) for item in list(dmin_ranges)]
rd['bstar_image'] = [round(item,4) for item in list(dmax_ranges)]

In [29]:
pkl(rd,'lml_measurement_ranges.pkl')